## installing libraries

In [1]:
# !pip install osmnx
# !pip install soundscapy
# !pip install geopandas
# !pip install matplotlib

In [2]:
# pip install ipykernel==5.5.6 ipython==7.34.0 pandas==1.5.3 'pyzmq<25'

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [3]:
# Add soundscapy to the Python path
# import sys
# sys.path.append('../..')
# imports
# %matplotlib inline
import soundscapy as sspy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
from scipy import stats
from scipy.stats import spearmanr
import geopandas as gpd


import warnings
# warnings.simplefilter('ignore')

%config InlineBackend.figure_format = 'svg'

# LOAD GDF

In [4]:
ssid = sspy.isd.load()
ssid = sspy.isd.remove_lockdown(ssid)
ssid, excl = sspy.isd.validate(ssid, allow_paq_na=False)
excl.head()

Renaming PAQ columns.
Checking PAQ data quality.
Identified 56 samples to remove.
[95, 108, 124, 127, 146, 154, 160, 178, 186, 203, 220, 226, 255, 381, 404, 407, 431, 562, 571, 577, 586, 590, 606, 631, 659, 666, 675, 689, 694, 707, 729, 839, 846, 877, 897, 992, 1007, 1027, 1055, 1056, 1064, 1097, 1214, 1217, 1222, 1223, 1225, 1229, 1235, 1255, 1268, 1269, 1300, 1316, 1321, 1335]


,LocationID,SessionID,GroupID,RecordID,Language,Lockdown,start_time,end_time,latitude,longitude,...,"FS_Avg,arith(vacil)","I_HM_Avg,arith(iu)","Ton_HM_Avg,arith(tuHMS)",LZeq_L(dB(SPL)),LAeq_L(A)(dB(SPL)),LA10_LA90(dB(SPL)),LCeq_LAeq(dB(SPL)),LC10_LC90(dB(SPL)),RA_2D_cp(cPa),PA(Zwicker)
95,CamdenTown,CamdenTown4,CT411,NaN,NaN,0,NaN,NaN,NaN,NaN,...,0.04410,0.369,0.608,85.82,75.37,4.79,10.00,6.34,18.8,54.379887
152,EustonTap,EustonTap1,ET102,NaN,NaN,0,NaN,NaN,NaN,NaN,...,0.02090,0.554,0.132,79.70,70.64,5.09,7.70,4.88,16.0,38.918730
168,EustonTap,EustonTap1,ET111,1300.0,English,0,28/10/2019 13:11,28/10/2019 13:14,51.5269,-0.1323,...,0.00591,0.316,0.242,77.47,69.26,3.30,7.25,4.18,13.5,33.043909
171,EustonTap,EustonTap1,ET114,NaN,NaN,0,NaN,NaN,NaN,NaN,...,0.01820,0.406,0.139,78.80,70.22,5.38,7.47,4.72,14.8,40.037874
190,EustonTap,EustonTap1,ET124,1324.0,English,0,28/10/2019 14:18,28/10/2019 14:19,51.5269,-0.1323,...,0.00761,0.383,0.201,77.56,69.89,3.25,6.87,3.12,15.1,40.247587


In [5]:
#@title create gdf
points_lon_lat = gpd.points_from_xy(ssid['longitude'], ssid['latitude'])
ssid_gdf = gpd.GeoDataFrame(ssid, geometry=points_lon_lat)
# ssid_gdf

In [6]:
# selecting only rows for filtered location (here location 1)
filter_location = ['CamdenTown']
ssid_gdf_location1 = ssid_gdf.loc[ssid['LocationID'].isin(filter_location)]
# ssid_gdf_location1.tail()

In [7]:
# Group by 'LocationID' and take the first 3 rows of each group
ssid_gdf_3_per_location = ssid_gdf.groupby('LocationID').head(3)  #.reset_index(drop=True)
ssid_gdf_1_per_location = ssid_gdf.groupby('LocationID').head(1)


# maps layout

## roads width

In [8]:
#@title Define the mapping of road types to levels
# used to later make destictions between roads
road_type_mapping = {
    'trunk': 0,
    'trunk_link': 0,
    'primary': 1,
    'primary_link': 1,
    'secondary': 2,
    'secondary_link': 2,
    'tertiary': 3,
    'tertiary_link': 3,
    'service': 3, #3to4 increased correlation by a loy
    'residential': 4,
    'unclassified': 4,
    'living_street': 4,
    'cycleway': 5,
    'footway': 5.5,
    'steps': 5.5,
    'pedestrian': 5.5

}

road_width = {
    'trunk': 4,
    'trunk_link': 4,
    'primary': 3,
    'primary_link':3,
    'secondary': 2,
    'secondary_link': 2,
    'residential': 2,
    'tertiary': 1.5,
    'tertiary_link': 1.5,
    'unclassified': 1,
    'living street': 1,
    'unclassified': 1,
    'service' : 1,
    'highway': 1,
    'unclassified': 1,
    'living_street': 1,
    'cycleway': 0.5,
    'footway': 0.5,
    'path' : 0.5,
    'steps': 0.5,
    'pedestrian': 0.5,
    'track' : 0.5,
    'corridor': 0.5
}

# Assigning a few default speed values (km/hour), fill in edges with missing `maxspeed` from OSM
hwy_speeds = {
    'trunk': 100,
    'trunk_link': 100,
    'primary': 80,
    'primary_link': 80,
    'secondary': 50,
    'secondary_link': 50,
    'tertiary': 30,
    'tertiary_link': 30,
    'residential': 30,
    'unclassified': 30,
    'living_street' : 15,
    'cycleway': 5,
    'footway': 5,
    'steps': 5,
    'pedestrian': 5
                }

# types of edges found in the dataset
#['footway', 'unclassified', 'primary', 'residential', 'pedestrian', 'cycleway', 'service', 'tertiary', 'trunk', 'secondary', 'steps', 'path', 'primary_link']

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# Assuming `road_width` is a predefined dictionary containing road types as keys and their respective widths as values

# Extracting road types
road_types = list(road_width.keys())

# Finding the road with the maximum and minimum width
max_width = max(road_width.values())
min_width = min(road_width.values())

# Define colors for normalization
base_color = np.array(mcolors.to_rgb('#ffffff'))  # White
highlight_color = np.array(mcolors.to_rgb('#eba834'))  # Orange-red

# Normalize road widths
normalized_widths = {road: (width - min_width) / (max_width - min_width) for road, width in road_width.items()}

# Assigning colors to roads based on their widths
road_colors = []
for road, width in normalized_widths.items():
    # Interpolating color based on the width
    color = (1 - width) * base_color + width * highlight_color
    road_colors.append(color)

# Printing the road types and their corresponding colors
print("Road Types:", road_types)
print("Road Colors:", road_colors)

# Creating a dictionary mapping road types to their colors
road_type_colors = dict(zip(road_types, road_colors))


Road Types: ['trunk', 'trunk_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'residential', 'tertiary', 'tertiary_link', 'unclassified', 'living street', 'service', 'highway', 'living_street', 'cycleway', 'footway', 'path', 'steps', 'pedestrian', 'track', 'corridor']
Road Colors: [array([0.92156863, 0.65882353, 0.20392157]), array([0.92156863, 0.65882353, 0.20392157]), array([0.94397759, 0.75630252, 0.43137255]), array([0.94397759, 0.75630252, 0.43137255]), array([0.96638655, 0.85378151, 0.65882353]), array([0.96638655, 0.85378151, 0.65882353]), array([0.96638655, 0.85378151, 0.65882353]), array([0.97759104, 0.90252101, 0.77254902]), array([0.97759104, 0.90252101, 0.77254902]), array([0.98879552, 0.9512605 , 0.88627451]), array([0.98879552, 0.9512605 , 0.88627451]), array([0.98879552, 0.9512605 , 0.88627451]), array([0.98879552, 0.9512605 , 0.88627451]), array([0.98879552, 0.9512605 , 0.88627451]), array([1., 1., 1.]), array([1., 1., 1.]), array([1., 1., 1.]), array([1

## tags OSM data

In [10]:
# tags for the OSM data
tags_buildings = {'building': True}

tags_natural = {'natural': ['fell', 'grassland', 'health', 'scrub', 'wood' ]} # {'natural': True }
tags_landuse = {'landuse':[ 'grass','greenfield']}
tags_parks = {'leisure':[ 'nature_reserve','park', 'track', 'garden' ]}
tags_water = {'natural': [ 'water' ]}
tags_wetland = {'natural': [ 'wetland' ]}
tags_tree = {'natural':[ 'grassland', 'tree', 'wood'] }


# main function

In [12]:
#get 5 random points from gdf
import random
random_rows = ssid_gdf.sample(n=5)

## def load osm data

In [13]:
#@title def load osm data
def load_osm_data(point, tags, size):
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    try:
        data = ox.features_from_point(point, tags, dist=size)
        return data
    except Exception as e:
        print(f'Error loading {tags} data: {e}')
        return None
    warnings.resetwarnings()



## def create bounding box from centerpoint

In [16]:
#@title def create bounding box from centerpoint
import math
import shapely.geometry as geometry

def degrees_to_meters(latitude):
    # Earth radius in meters
    earth_radius = 6378137

    # Convert latitude from degrees to radians
    lat_rad = math.radians(latitude)

    # Calculate the length of one degree of latitude in meters
    lat_length = math.cos(lat_rad) * math.pi * earth_radius / 180

    # Calculate the length of one degree of longitude in meters
    lon_length = math.pi * earth_radius / 180

    return lat_length, lon_length


def meters_to_degrees(point, meters):
    latitude = point[0]
    longitude = point[1]
     # Get the length of one degree of latitude and longitude
    lat_length, lon_length = degrees_to_meters(latitude)

    # Convert meters to degrees
    lat_degrees = meters / lat_length
    lon_degrees = meters / lon_length

    return lat_degrees, lon_degrees


def get_bounding_box(center_point, meters):
    latitude = center_point[0]
    longitude = center_point[1]

    # Calculate how many degrees correspond to the given meters
    lat_degrees, lon_degrees = meters_to_degrees(center_point, meters)

    # Calculate the bounding box coordinates
    y_min = latitude - lon_degrees
    y_max = latitude + lon_degrees
    x_min = longitude - lat_degrees
    x_max = longitude + lat_degrees

    # Create a shapely polygon representing the bounding box
    bounding_box = geometry.box(x_min, y_min, x_max, y_max)

    # Return the bounding box and its coordinates
    return bounding_box, [y_max, y_min, x_max, x_min]

# Example usage:
# center_point = (40.730610, -73.935242)  # Example center point (latitude, longitude)
meters = 300  # Example distance in meters

# bounding_box, bbox_coordinates = get_bounding_box(center_point, meters)
# print("Bounding Box Coordinates:", bbox_coordinates)
# print("Bounding Box WKT:", bounding_box.wkt)


## def create scalebar

In [17]:
#@title def create scalebar
from shapely.geometry import box

def create_scale_bar(bbox_coordinates, center_point, meters=4):
    """
    Generates a scale bar represented by a list of shapely polygons
    within the given bounding box coordinates.

    Parameters:
    - bbox_coordinates (list): A list of bounding box coordinates [y_max, y_min, x_max, x_min].
    - meters (float): Length in meters for the scale bar.

    Returns:
    - scalebar_list (list): List of shapely polygons representing the scale bar.
    - bbox_coords_updated (list): Updated bounding box coordinates.
    """

    # Convert meters to degrees for latitude and longitude
    lat_degrees, lon_degrees = meters_to_degrees(center_point, meters)

    # Extracting the lower right corner coordinates
    y_min, x_min = bbox_coordinates[1] + lon_degrees, bbox_coordinates[3] + lat_degrees

    # Calculate new bounding box coordinates
    y_max = y_min + lon_degrees / 2
    x_max = x_min + lat_degrees

    # Generate shapely polygons for the bounding boxes
    scalebar_list = []
    for i in range(3):
        x_min_i = x_min + (lat_degrees ) * i
        x_max_i = x_min + (lat_degrees) * (i + 1)
        scalebar_list.append(box(x_min_i, y_min, x_max_i, y_max))

    # Update bounding box coordinates
    bbox_coords_updated = [y_max, y_min, x_max, x_min]

    return scalebar_list, bbox_coords_updated


## create polygon radius

In [18]:
#@title def create polygon radius
import shapely
from shapely import Point

def create_radius_from_bbox(center_point, bounding_box, dist_plot):

    lat_degrees, lon_degrees = meters_to_degrees(center_point, dist_plot)

    #create ellipse from circle from point
    center_point_point = Point(center_point[1], center_point[0])
    circle = center_point_point.buffer(lon_degrees)
    ellipse = shapely.affinity.scale(circle, lat_degrees/lon_degrees, 1)

    #subtract the ellipse from the bbox
    difference = bounding_box.difference(ellipse)

    return difference



## def plotting points roads, vegetation, water

In [ ]:
#@title def plotting points roads, vegetation, water
import osmnx as ox
import matplotlib.pyplot as plt

def plot_from_multiple_points(index, G, edges_gdf02_exploded, buildings_osm, park_natural,
                              landuse_natural, water_natural, wetland_natural, tree_natural,
                              center_point, scalebar_polygons, multipolygon_radius, text
                              fig_size=[10,10], dpi=600, bbox=None):
    """
    Plot the road network, buildings, and natural features from multiple data sets.

    Parameters:
        - G: Graph representing the road network.
        - edges_gdf02_exploded: GeoDataFrame representing exploded edges.
        - buildings_osm: GeoDataFrame representing buildings.
        - park_natural: GeoDataFrame representing parks.
        - landuse_natural: GeoDataFrame representing land use.
        - water_natural: GeoDataFrame representing water bodies.
        - tree_natural: GeoDataFrame representing trees.
        - center_point: Tuple representing the center point for scatter plot.
        - fig_size: Size of the figure (default=[15,15]).
        - dpi: Dots per inch for the figure resolution (default=600).
    """
    # Plot the road network and exploded edges if G is not None
    if G is not None:
        fig, ax = ox.plot_graph(G, figsize=fig_size, bgcolor='lightgrey', node_color='red', node_size=10,
                                node_alpha=0, node_edgecolor='none', node_zorder=1, edge_color='xkcd:pastel red',
                                edge_linewidth=2, edge_alpha=0.3, show=False, close=False, save=False,
                                filepath=None, dpi=dpi, bbox=bbox)

    # edges_gdf02_exploded.plot(ax=ax, color='white', linewidth=edges_gdf02_exploded['road_width']*4, alpha=0.2, legend=True)
    edges_gdf02_exploded.plot(ax=ax, color=edges_gdf02_exploded['edge_color_roadtype'],
                              linewidth=edges_gdf02_exploded['road_width']*8, alpha=0.5, legend=True)

    # Plot buildings on the same axis if buildings_osm is not None
    if buildings_osm is not None:
        fig, ax = ox.plot_footprints(buildings_osm, ax=ax, figsize=fig_size, color='white', edge_color='black',
                                     edge_linewidth=1, alpha=0.8,
                                     bgcolor='grey', bbox=bbox, save=False, show=False, close=False, filepath=None, dpi=dpi)

    # Scatter plot for center point
    plt.scatter(center_point[1], center_point[0], color='red', label='Center Point', zorder=3)

    for geom in scalebar_polygons:
      xs, ys = geom.exterior.xy
      ax.fill(xs, ys, alpha=1, fc='black', ec='white', zorder= 4)

    radius = gpd.GeoSeries(multipolygon_radius)
    radius.plot(ax=ax, color = 'white', zorder=3 )
    plt.text(bbox[0], bbox[2], text)

    # Define data sets for natural features
    data_sets = [
        (park_natural, 'darkseagreen', 50, 'park_natural', 0.5, 2),
        (vegetation_natural, 'purple', 0.5, 'vegetation',0.1, 1), #natural all
        (landuse_natural, 'yellowgreen', 50, 'landuse_natural', 0.5, 1), #grass
        (water_natural, 'teal', 1, 'water_natural', 0.5, 1),
        (wetland_natural, 'teal', 1, 'tan', 0.1, 1),
        (tree_natural, 'olivedrab', 150, 'tree_natural', 0.7, 2),
        (tree_natural, 'darkolivegreen', 2.5, 'tree_natural', 0.5, 2) #double to create a 'stem'
    ]

    # Plot each data set
    for data, color, size, label, alpha, zorder in data_sets:
        if data is not None:
            data.plot(ax=ax, markersize=size, color=color, label=label, alpha=alpha, zorder=zorder)

    # temp_file_path = f'240226_point_{index}_plot.png'
    # plt.savefig(temp_file_path)
    # files.download(temp_file_path) # # Download the temporary file to the local computer

    ## GOOGLE COLAB
    # Define the file path within the folder
    # temp_file_path = os.path.join(folder_path, f'240226_sample_point_{index}_plot.png')
    # Save the plot directly to the specified folder
    # plt.savefig(temp_file_path)
    
    ## VSCODE
    # Construct the filename string
    filename = f"240228_plot_{index}.png"
    # Save the plot to the specified folder with the constructed filename
    plt.savefig(r'C:/Users/n.smit/OneDrive - Oosterhoff Group/Documents/Master Thesis/DATA/plot_maps/' + filename)

    plt.legend(loc='upper right')
    plt.show()
    plt.close(fig)

## def to load data for each point

In [22]:
#@title def to load data for each point
def load_data_for_point(row, distance=100):

    point = (row['latitude'], row['longitude'])
    size = distance + 50
    distance = distance + 50

    # Fetch street network
    G = ox.graph_from_point(point, dist=distance, network_type='all')
    # edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
    # new_network = ox.graph_from_gdfs(nodes_gdf, edges_gdf)

    # Preprocess street network data
    # edges_gdf = preprocess_street_network(G)

    # Load natural features
    vegetation_natural = load_osm_data(point, tags_natural, size=size)  #Natural : True
    landuse_natural_grass = load_osm_data(point, tags_landuse, size=size)
    park_natural = load_osm_data(point, tags_parks, size=size)
    water_natural = load_osm_data(point, tags_water, size=size)
    wetland_natural = load_osm_data(point, tags_wetland, size=size)
    tree_natural = load_osm_data(point, tags_tree, size=size-40)

    # Load buildings
    buildings_osm = load_osm_data(point, tags_buildings, size=size)

    return G, vegetation_natural, landuse_natural_grass, park_natural, water_natural, wetland_natural,  tree_natural, buildings_osm

# Function to preprocess street network data
def preprocess_street_network(G):
    edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
    edges_gdf_exploded = edges_gdf.explode('highway')
    edges_gdf_exploded['road_width'] = edges_gdf_exploded['highway'].map(road_width).fillna(3)
    edges_gdf_exploded['edge_color_roadtype'] = edges_gdf_exploded['highway'].map(road_type_colors).fillna('purple')
    return edges_gdf_exploded

## function for loading the images

In [ ]:
#@title function for loading the images
# from google.colab import files

distance = 70
meters = 1
# Calculate how many degrees correspond to 1 meters


# Now, unique values can be extracted without any lists
unique_values_lists = []
print(unique_values_lists)


# Loop through each point
for index, row in ssid_gdf_1_per_location.iterrows():
    print('Processing data for Point', index, 'in',  row['LocationID'])
    center_point = (row['latitude'], row['longitude'])
    lat_degrees, lon_degrees = meters_to_degrees(center_point, meters)

    bounding_box, bbox_coordinates = get_bounding_box(center_point, distance)
    scalebar_polygons, scale_coordinates  = create_scale_bar(bbox_coordinates, center_point, meters = 4)
    multipolygon_radius = create_radius_from_bbox(center_point, bounding_box, distance)

    # Load data for the current point
    network, vegetation_natural, landuse_natural, park_natural, water_natural, wetland_natural, tree_natural, buildings_osm = load_data_for_point(row, distance)
    edges_gdf = preprocess_street_network(network)

    # park_natural = create_inner_polygons(park_natural, lat_degrees*5)
    unique_values_list = edges_gdf['highway'].unique().tolist()
    unique_values_lists.append(unique_values_list)
    print(unique_values_lists)
    # Plotting
    plot_from_multiple_points(index, network, edges_gdf, buildings_osm, park_natural, landuse_natural,
                              water_natural, wetland_natural, tree_natural, center_point,
                              scalebar_polygons, multipolygon_radius,
                              fig_size=[10,10], dpi=600, bbox=bbox_coordinates)

    # # Save the plot as an image
    # plt.savefig(f'point_{index}_plot.png')
    # plt.close(fig)
    # Save the plot to a temporary file



In [ ]:
# Define tags for natural features excluding trees
tags_natural_exclude_trees = {'natural': ['fell', 'grassland', 'health', 'scrub', 'wood' ]}

vegetation_natural = load_osm_data(center_point, tags_natural_exclude_trees, size=300)
# Load OSMnx data with natural features excluding trees
# gdf_filtered_natural = ox.geometries_from_place(place_name, tags=tags_natural_exclude_trees)


Error loading {'natural': ['fell', 'grassland', 'health', 'scrub', 'wood']} data: [Errno 107] Transport endpoint is not connected: 'cache/91edd4dbc07d1dc706f6231228d6cb67793a872c.json'


In [ ]:
import geopandas as gpd

# Assuming vegetation_natural is your GeoDataFrame

# Calculate the area of each geometry (assuming they are polygons)
vegetation_natural['area'] = vegetation_natural.geometry.area

# Sort the GeoDataFrame based on the area
vegetation_natural_sorted = vegetation_natural.sort_values(by='area', ascending=False)

# Remove the area column if you don't need it anymore
vegetation_natural_sorted.drop(columns=['area'], inplace=True)


In [ ]:
vegetation_natural

#Load OSM roads
 graphs

## roads

In [ ]:
# Convert the GeoDataFrame to a list of points
points_lon_lat = list(zip(ssid_gdf['latitude'], ssid_gdf['longitude']))

# Get the center point of the GeoDataFrame
center_point = points_lon_lat[0]  # Considering the first row as the center point
distance = 300  # in meters
# Fetch the street network
G = ox.graph_from_point(center_point, dist=distance, network_type='all')
print(center_point)

###colors for the roads

In [ ]:
G = ox.distance.add_edge_lengths(G, edges=None)
G = ox.add_edge_speeds(G, hwy_speeds)
basic_stats_G = ox.stats.basic_stats(G, area=None, clean_int_tol=None)
basic_stats_G

## edge colors

create a gdf from the road network

In [ ]:
#create gdfs for the nodes and edges. Create a newwork from that. Create new gdfs from this one
nodes_gdf00, edges_gdf00 = ox.graph_to_gdfs(G, nodes=True, edges=True)
new_network = ox.graph_from_gdfs(nodes_gdf00, edges_gdf00)

# Create the geodataframes nodes_gdf and edges_gdf
nodes_gdf02, edges_gdf02 = ox.graph_to_gdfs(new_network, nodes=True, edges=True)

In [ ]:
edges_gdf00.columns

In [ ]:
#explode the column to get all the values
edges_gdf02_exploded = edges_gdf02.explode('highway')

# Now, unique values can be extracted without any lists
unique_values_list = edges_gdf02_exploded['highway'].unique().tolist()
print(unique_values_list)


## add colors and width


In [ ]:
# Create a new column for edge width based on road type
edges_gdf02_exploded['road_width'] = edges_gdf02_exploded['highway'].map(road_width).fillna(1)
# Create a new column for edge colors based on road type
edges_gdf02_exploded['edge_color_roadtype'] = edges_gdf02_exploded['highway'].map(road_type_colors).fillna('purple')

In [ ]:
# Plotting the road network with the corresponding edge colors and edge widths
fig_size = [10,10]

fig, ax = plt.subplots(figsize=fig_size)
edges_gdf02_exploded.plot(ax=ax,  color=edges_gdf02_exploded['edge_color_roadtype'],linewidth=edges_gdf02_exploded['road_width']*2, alpha = 0.5, legend=True)

# Add a legend for the edge colors
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=road_type, markerfacecolor=color)
                   for road_type, color in road_type_colors.items()]
plt.legend(handles=legend_elements, title='Road Types')
ax.set_title('Road Types ')

# Display the plot
plt.axis('on')
plt.show()


## folium

In [ ]:
import folium

# Create a map centered at the defined point
mymap = folium.Map(location=center_point, zoom_start=15)

# Example: Add a marker at the center point
folium.Marker(center_point, popup='Center Point').add_to(mymap)

for i in points_lon_lat:
  folium.Marker(i, popup='Center Point').add_to(mymap)

folium.LayerControl().add_to(mymap)
# Display the map
mymap.save('mymap.html')  # Save the map to an HTML file
# mymap

# Load OSM vegetation + buildings

In [ ]:
#load vegetation
vegetation_natural = load_osm_data(center_point, tags_natural, size = 300 )
landuse_natural = load_osm_data(center_point, tags_landuse, size = 300 )
park_natural = load_osm_data(center_point, tags_parks, size = 300 )
water_natural = load_osm_data(center_point, tags_water, size = 300)
tree_natural = load_osm_data(center_point, tags_tree, size = 300 )

#load buildings
buildings_osm = load_osm_data(center_point, tags_buildings, size = 400)

In [ ]:
longitude = center_point[1]
latitude = center_point[0]
# meters = distance
meters = 1
# Calculate how many degrees correspond to 50 meters
lat_degrees, lon_degrees = meters_to_degrees(center_point, meters)

print("At latitude {}, {} meters correspond to approximately {} degrees of latitude and {} degrees of longitude.".format(latitude, meters, lat_degrees, lon_degrees))


At latitude 51.539124, 1 meters correspond to approximately 1.4442830144070832e-05 degrees of latitude and 8.983152841195214e-06 degrees of longitude.


In [ ]:
dist_plot = 300

bounding_box, bbox_coordinates = get_bounding_box(center_point, dist_plot)
scalebar_polygons, scale_coordinates  = create_scale_bar(bbox_coordinates, center_point, meters = 4)
multipolygon_radius = create_radius_from_bbox(center_point, bounding_box, dist_plot)


In [ ]:
gdf_trunk = edges_gdf02_exploded[edges_gdf02_exploded['highway']== 'trunk']
gdf_primary = edges_gdf02_exploded[edges_gdf02_exploded['highway']== 'primary']

buffer_trunk = gdf_trunk.buffer(lat_degrees/10)

gdf_trunk.plot()
# gdf_primary.plot()

#plot all together

In [ ]:
fig_size= [10,10]
dpi=600
if G is not None:
# Plot the road network
    fig, ax = ox.plot_graph(G, figsize=fig_size, bgcolor='lightgrey', node_color='red', node_size=10,
                        node_alpha=0, node_edgecolor='none', node_zorder=1, edge_color='xkcd:pastel red',
                        edge_linewidth=1, edge_alpha=0.2, show=False, close=False, save=False, filepath=None,
                            dpi=dpi, bbox=bbox_coordinates)

edges_gdf02_exploded.plot(ax=ax, color='white',linewidth=edges_gdf02_exploded['road_width']*4, alpha = 0.2, legend=True)

edges_gdf02_exploded.plot(ax=ax, color=edges_gdf02_exploded['edge_color_roadtype'],linewidth=edges_gdf02_exploded['road_width']*4,
                          alpha = 0.6, legend=True)

if buildings_osm is not None:
    # Plot buildings on the same axis
    fig, ax = ox.plot_footprints(buildings_osm, ax=ax, figsize=fig_size, color='white', edge_color='black', edge_linewidth=1, alpha=0.7,
                                bgcolor='grey', bbox=bbox_coordinates, save=False, show=False, close=False, filepath=None, dpi=dpi)

# ssid_gdf.plot(ax=ax, cmap='cool', alpha=0.7, markersize=50, legend=True)
plt.scatter(center_point[1], center_point[0], color='red', label='Center Point', zorder =2)


for geom in scalebar_polygons:
  xs, ys = geom.exterior.xy
  ax.fill(xs, ys, alpha=1, fc='black', ec='white', zorder= 4)

radius = gpd.GeoSeries(multipolygon_radius)
radius.plot(ax=ax, color = 'white', zorder=3 )
# for geom in difference:
#   x,y = geom.exterior.xy
#   plt.plot(x,y)
# p = gpd.GeoSeries(poly)
# p.plot(ax=ax, color = 'purple', alpha = 1)
#  plt.show()
# poly.plot(ax=ax, color = 'purple', alpha = 0.2, edge_color = 'blue')
data_sets = [
    (park_natural, 'darkseagreen', 50, 'park_natural', 0.1),
    #(vegetation_natural, 'forestgreen', 50, 'vegetation', 1),
    (landuse_natural, 'yellowgreen', 50, 'landuse_natural', 0.1),
    (water_natural, 'seagreen', 1, 'water_natural', 0.5),
    (tree_natural, 'olivedrab', 100, 'tree_natural', 0.5),
    (tree_natural, 'darkolivegreen', 1, 'tree_natural', 0.5) #double to create a 'stem'
]

for data, color, size, label, alpha in data_sets:
    if data is not None:
        data.plot(ax=ax, markersize=size, color=color, label=label, alpha=alpha, zorder = 2)

In [ ]:
filter_location = ['MonumentoGaribaldi']
ssid_gdf_locationxx = ssid_gdf.loc[ssid['LocationID'].isin(filter_location)]
points_lon_latxx = list(zip(ssid_gdf_locationxx['latitude'], ssid_gdf_locationxx['longitude']))

# Get the center point of the GeoDataFrame
center_point = points_lon_latxx[0]
park_natural = load_osm_data(center_point, tags_parks, size = 300 )

In [ ]:
def create_inner_polygon(row, buffer_distance):
    # print(row)
    outer_polygon = row
    inner_polygons = []

    # Check if the geometry is a MultiPolygon
    if outer_polygon.geom_type == 'MultiPolygon':
        print(outer_polygon)
        polygons = list(outer_polygon.geoms)
        # Iterate over each polygon in the MultiPolygon
        for polygon in polygons:
            print(polygon)
            inner_polygon = polygon.buffer(-buffer_distance)
            # Append the inner polygon to the list
            inner_polygons.append(inner_polygon)
    else:
        # For single polygons, directly buffer inward
        inner_polygon = outer_polygon.buffer(-buffer_distance)
        inner_polygons.append(inner_polygon)

    # Find the largest inner polygon (by area) if there are multiple
    if inner_polygons:
        largest_inner_polygon = max(inner_polygons, key=lambda polygon: polygon.area)
        return largest_inner_polygon
    else:
        # Return None if there are no inner polygons
        return None




def create_inner_polygons(park_natural, buffer_distance):
    """
    Function to create inner polygons for each polygon in a GeoDataFrame.

    Args:
    park_natural (GeoDataFrame): GeoDataFrame containing polygon geometries.
    buffer_distance (float): Distance to buffer the polygons inward to create inner polygons.

    Returns:
    GeoDataFrame: GeoDataFrame containing both the original polygons and the inner polygons.
    """
    # # Function to create inner polygon for each row in the GeoDataFrame
    # def create_inner_polygon(row, buffer_distance):
    #     outer_polygon = row
    #     inner_polygon = outer_polygon.buffer(-buffer_distance)  # Buffer with a negative distance to create inner polygon
    #     if inner_polygon.geom_type == 'MultiPolygon':
    #         inner_polygon = max(inner_polygon, key=lambda polygon: polygon.area)
    #     return inner_polygon

    # Create inner polygons
    # print(park_natural.geom_type)
    print(park_natural)
    if park_natural is not None:
        inner_polygons = gpd.GeoDataFrame(geometry=park_natural.geometry.apply(create_inner_polygon, args=(buffer_distance,)), crs=park_natural.crs)
        inner_polygons01 = gpd.GeoDataFrame(geometry=inner_polygons.geometry.apply(create_inner_polygon, args=(buffer_distance,)), crs=park_natural.crs)

        # Concatenate inner_polygons and inner_polygons01
        combined_inner_polygons = pd.concat([inner_polygons, inner_polygons01])

        # Add a column to identify the type of inner polygon (if needed)
        combined_inner_polygons['inner_polygon_type'] = ['inner_polygons']*len(inner_polygons) + ['inner_polygons01']*len(inner_polygons01)

        # Append combined inner polygons to the original GeoDataFrame
        park_natural_with_inner = pd.concat([park_natural, combined_inner_polygons]).reset_index(drop=True)

    else:
        park_natural_with_inner = park_natural

    return park_natural_with_inner

# Example usage:
# park_natural_gdf is your GeoDataFrame containing the polygons
# buffer_distance is the distance to buffer inward to create inner polygons
park_natural_with_inner = create_inner_polygons(park_natural, lat_degrees*2)
# Plotting for visualization
fig, ax = plt.subplots()
park_natural_with_inner.plot(ax=ax, color='lightgreen', edgecolor= None, alpha=0.3)

# load data per location

## def to load data for each point

In [30]:
#@title def to load data for each point
def load_data_for_center_point(point, distance=50):

    # point = (row['latitude'], row['longitude'])
    size = distance + 50
    distance = distance + 50

    # Fetch street network
    G = ox.graph_from_point(point, dist=distance, network_type='all')
    # edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
    # new_network = ox.graph_from_gdfs(nodes_gdf, edges_gdf)

    # Preprocess street network data
    # edges_gdf = preprocess_street_network(G)

    # Load natural features
    vegetation_natural = load_osm_data(point, tags_natural, size=size)  #Natural : True
    landuse_natural_grass = load_osm_data(point, tags_landuse, size=size)
    park_natural = load_osm_data(point, tags_parks, size=size)
    water_natural = load_osm_data(point, tags_water, size=size)
    wetland_natural = load_osm_data(point, tags_wetland, size=size)
    tree_natural = load_osm_data(point, tags_tree, size=size-40)

    # Load buildings
    buildings_osm = load_osm_data(point, tags_buildings, size=size)

    return G, vegetation_natural, landuse_natural_grass, park_natural, water_natural, wetland_natural,  tree_natural, buildings_osm

# Function to preprocess street network data
def preprocess_street_network(G):
    edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
    edges_gdf_exploded = edges_gdf.explode('highway')
    edges_gdf_exploded['road_width'] = edges_gdf_exploded['highway'].map(road_width).fillna(3)
    edges_gdf_exploded['edge_color_roadtype'] = edges_gdf_exploded['highway'].map(road_type_colors).fillna('purple')
    return edges_gdf_exploded

In [ ]:
from scipy.stats import zscore
import numpy as np

def replace_outliers_w_zero(gdf, threshold=1, null_object=np.nan):
    """
    Replace outliers in a GeoDataFrame with a null object based on z-scores of latitude and longitude.

    Parameters:
    - gdf: GeoDataFrame with 'latitude' and 'longitude' columns.
    - threshold: Z-score threshold for identifying outliers (default is 3).
    - null_object: Value to replace outliers with (default is np.nan).

    Returns:
    - gdf_no_outliers: GeoDataFrame with outliers replaced by null_object.
    - outliers: DataFrame containing the identified outliers.
    - outliers_count: Number of outliers replaced.
    """
    latitudes = gdf['latitude']
    longitudes = gdf['longitude']

    # Calculate z-scores for latitudes and longitudes
    z_scores_lat = zscore(latitudes)
    z_scores_lon = zscore(longitudes)

    # Identify outliers based on either latitude or longitude
    outliers_mask = (abs(z_scores_lat) > threshold) | (abs(z_scores_lon) > threshold)

    # Create a copy of the GeoDataFrame to avoid modifying the original data
    gdf_no_outliers = gdf.copy()

    # Replace outliers with null_object
    # gdf_no_outliers.loc[outliers_mask, ['latitude', 'longitude']] = 0

    gdf_no_outliers['outlier'] = outliers_mask

    # Create a DataFrame containing the identified outliers
    outliers = gdf_no_outliers[outliers_mask]
    gdf_remove_outliers = gdf_no_outliers[~outliers_mask]

    # Count of outliers replaced
    outliers_count = outliers_mask.sum()

    return gdf_no_outliers, outliers, outliers_count

def calculate_center_point(gdf):
    """
    Calculate the center point based on the median latitude and longitude of a GeoDataFrame.

    Parameters:
    - gdf: GeoDataFrame with 'latitude' and 'longitude' columns.

    Returns:
    - center_point: Tuple containing the latitude and longitude of the center point.
    """

    # Remove outliers from the original GeoDataFrame
    gdf = gdf[gdf['outlier'] != True]
    latitudes = gdf['latitude']
    longitudes = gdf['longitude']


    # Calculate the median of latitudes and longitudes
    latitude_median = latitudes.median()
    longitude_median = longitudes.median()

    # Create a center point tuple
    center_point = (latitude_median, longitude_median)
    center_point_point = gdf.dissolve().centroid

    return center_point, center_point_point

def is_nearby(point, central_point, distance=lat_degrees*100):
    value = point.distance(central_point) <= distance
    # print(value)
    return value

outliers_counts = []
groups = []
groups_no_outliers = []

for location_id, group in ssid_gdf.groupby('LocationID'):
    print('calculating for', location_id)
    group_no_outliers, outliers, outliers_count = replace_outliers_w_zero(group)


    center_point, center_point_point = calculate_center_point(group_no_outliers)
    # print(center_point)
    group['nearby'] = group.apply(lambda row: is_nearby(row['geometry'],
                                                                    center_point_point), axis=1)

    for index, row in g.iterrows():
    G, vegetation_natural, landuse_natural, park_natural, water_natural, wetland_natural, tree_natural, buildings_osm = load_data_for_center_point(center_point)

    edges_gdf = preprocess_street_network(G)

    # lat_degrees, lon_degrees = meters_to_degrees(center_point, meters)
    bounding_box, bbox_coordinates = get_bounding_box(center_point, distance)
    scalebar_polygons, scale_coordinates  = create_scale_bar(bbox_coordinates, center_point, meters = 4)
    multipolygon_radius = create_radius_from_bbox(center_point, bounding_box, distance)

    # Plotting
    plot_from_multiple_points(index, G, edges_gdf, buildings_osm, park_natural, landuse_natural,
                              water_natural, wetland_natural, tree_natural, center_point,
                              scalebar_polygons, multipolygon_radius,
                              fig_size=[10,10], dpi=600, bbox=bbox_coordinates)
    outliers_counts.append(outliers_count)
    groups.append(group)
    groups_no_outliers.append(group_no_outliers)


# Print the number of outliers removed
print(f"\nNumber of outliers replaced: {outliers_counts}")

# # Print the outliers
# print("Outliers:")
# print(outliers)
group_no_outliers = groups_no_outliers[0]
group_no_outliers[group_no_outliers['outlier'] == True]
group01 = groups[7]
group01[group01['nearby'] == False]

# svm


In [ ]:
import os
from PIL import Image
# Directory containing the images
directory = '/content/drive/MyDrive/Colab Notebooks/240226_allpoints/sample_1/'
# List all image files in the directory
image_files = [file for file in os.listdir(directory) if file.endswith(('jpg', 'png', 'jpeg'))]

# Load images
images = []
for filename in image_files:
    image_path = os.path.join(directory, filename)
    image = Image.open(image_path)
    images.append(image)

print(images)

In [ ]:
ssid_gdf_3_per_location

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Preprocess and extract features from your image and geospatial data
# Assuming you have extracted features and stored them in X_image and X_geospatial, and labels in y

# Convert images list to a numpy array
X_image = np.array(images)

# Assuming 'X_geospatial' is already a 1-dimensional numpy array
X_geospatial = ssid_gdf_3_per_location['ISOPleasant'].values
# Assuming X_image is a numpy array and X_geospatial is a list
X_geospatial_array = np.array(X_geospatial)

# Concatenate image and geospatial features
X_combined = np.concatenate((X_image, X_geospatial_array.reshape(-1, 1)), axis=1)
# X_geospatial = ssid_gdf_3_per_location['ISOPleasant'].reset_index(drop=True)
# Concatenate image and geospatial features
# X_combined = np.concatenate((X_image, X_geospatial.reshape(-1, 1)), axis=1)
# 2. Concatenate image and geospatial features
# X_combined = np.concatenate((X_image, X_geospatial), axis=1)




# 3. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# 4. Train the SVM model
svm_model = svm.SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

# 5. Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# Convert X_geospatial to a numpy array
X_geospatial_array = np.array(X_geospatial)

# Ensure X_image is already a numpy array

# Check shapes
print("X_image shape:", X_image.shape)
print("X_geospatial_array shape:", X_geospatial_array.shape)

# Reshape X_geospatial_array if necessary
# Reshape it to have 39 rows and 1 column
X_geospatial_array = X_geospatial_array.reshape(-1, 1)

# Concatenate image and geospatial features
X_combined = np.concatenate((X_image, X_geospatial_array), axis=1)

In [ ]:
print( X_image[0:1])

# new

In [27]:
from scipy.stats import zscore
import numpy as np

def replace_outliers_w_zero(gdf, threshold=1, null_object=np.nan):
    """
    Replace outliers in a GeoDataFrame with a null object based on z-scores of latitude and longitude.

    Parameters:
    - gdf: GeoDataFrame with 'latitude' and 'longitude' columns.
    - threshold: Z-score threshold for identifying outliers (default is 3).
    - null_object: Value to replace outliers with (default is np.nan).

    Returns:
    - gdf_no_outliers: GeoDataFrame with outliers replaced by null_object.
    - outliers: DataFrame containing the identified outliers.
    - outliers_count: Number of outliers replaced.
    """
    latitudes = gdf['latitude']
    longitudes = gdf['longitude']

    # Calculate z-scores for latitudes and longitudes
    z_scores_lat = zscore(latitudes)
    z_scores_lon = zscore(longitudes)

    # Identify outliers based on either latitude or longitude
    outliers_mask = (abs(z_scores_lat) > threshold) | (abs(z_scores_lon) > threshold)

    # Create a copy of the GeoDataFrame to avoid modifying the original data
    gdf_no_outliers = gdf.copy()

    # Replace outliers with null_object
    # gdf_no_outliers.loc[outliers_mask, ['latitude', 'longitude']] = 0

    gdf_no_outliers['outlier'] = outliers_mask

    # Create a DataFrame containing the identified outliers
    outliers = gdf_no_outliers[outliers_mask]
    gdf_remove_outliers = gdf_no_outliers[~outliers_mask]

    # Count of outliers replaced
    outliers_count = outliers_mask.sum()

    return gdf_no_outliers, outliers, outliers_count

def calculate_center_point(gdf):
    """
    Calculate the center point based on the median latitude and longitude of a GeoDataFrame.

    Parameters:
    - gdf: GeoDataFrame with 'latitude' and 'longitude' columns.

    Returns:
    - center_point: Tuple containing the latitude and longitude of the center point.
    - center_point_point: Shapely point of the center_point
    """

    # Remove outliers from the original GeoDataFrame
    gdf = gdf[gdf['outlier'] != True]
    latitudes = gdf['latitude']
    longitudes = gdf['longitude']


    # Calculate the median of latitudes and longitudes
    latitude_median = latitudes.median()
    longitude_median = longitudes.median()

    # Create a center point tuple
    center_point = (latitude_median, longitude_median)
    center_point_point = gdf.dissolve().centroid

    return center_point, center_point_point

def is_nearby(point, central_point, distance=lat_degrees*100):
    value = point.distance(central_point) <= distance
    # print(value)
    return value

outliers_counts = []
groups = []
groups_no_outliers = []

# for location_id, group in ssid_gdf.groupby('LocationID'):
#     print('calculating for', location_id)
#     group_no_outliers, outliers, outliers_count = replace_outliers_w_zero(group)


#     center_point, center_point_point = calculate_center_point(group_no_outliers)
#     # print(center_point)
#     group['nearby'] = group.apply(lambda row: is_nearby(row['geometry'],
#                                                                     center_point_point), axis=1)

#     G, vegetation_natural, landuse_natural, park_natural, water_natural, wetland_natural, tree_natural, buildings_osm = load_data_for_center_point(center_point)
#     edges_gdf = preprocess_street_network(G)

#     for index, row in group.iterrows():
#         print('Processing data for Point', index, 'in',  row['LocationID'])
#         #create stuff per point
#         point_row =  (row['latitude'], row['longitude'])
#         bounding_box, bbox_coordinates = get_bounding_box(point_row, distance)
#         scalebar_polygons, scale_coordinates  = create_scale_bar(bbox_coordinates, point_row, meters = 4)
#         multipolygon_radius = create_radius_from_bbox(point_row, bounding_box, distance)
#         # check if the point is nearby
#         if row['nearby'] is True:
#             point_row =  (row['latitude'], row['longitude'])
#             # Plotting
#             plot_from_multiple_points(index, G, edges_gdf, buildings_osm, park_natural, landuse_natural,
#                                   water_natural, wetland_natural, tree_natural, point_row,
#                                   scalebar_polygons, multipolygon_radius,
#                                   fig_size=[10,10], dpi=600, bbox=bbox_coordinates)
#         else:
#             point_row =  (row['latitude'], row['longitude'])
#             center_point = point_row
#             G_temp, vegetation_temp, landuse_temp, park_temp, wate_temp, wetland_temp, tree_temp, buildings_osm_temp = load_data_for_center_point(center_point)
#             edges_gdf_temp = preprocess_street_network(G_temp)
#             plot_from_multiple_points(index, G_temp, edges_gdf_temp, buildings_osm_temp, park_temp, landuse_temp,
#                                   water_temp, wetland_temp, tree_temp, point_row,
#                                   scalebar_polygons, multipolygon_radius,
#                                   fig_size=[10,10], dpi=600, bbox=bbox_coordinates)




#     outliers_counts.append(outliers_count)
#     groups.append(group)
#     groups_no_outliers.append(group_no_outliers)


# Print the number of outliers removed
print(f"\nNumber of outliers replaced: {outliers_counts}")

# # Print the outliers
# print("Outliers:")
# print(outliers)
# group_no_outliers = groups_no_outliers[0]
# group_no_outliers[group_no_outliers['outlier'] == True]
# group01 = groups[7]
# group01[group01['nearby'] == False]


Number of outliers replaced: []


In [32]:
LocationID = "Location123"
index = 1

# Construct the filename string
filename = f"{LocationID}_plot_{index}.png"

# Save the plot to the specified folder with the constructed filename
plt.savefig(r'C:/Users/n.smit/OneDrive - Oosterhoff Group/Documents/Master Thesis/DATA/plot_maps/' + filename)


<Figure size 640x480 with 0 Axes>

In [ ]:
import pandas as pd

def process_location_data(ssid_gdf, distance=100):
    outliers_counts = []
    groups = []
    groups_no_outliers = []

    for location_id, group in ssid_gdf.groupby('LocationID'):
        print(f'Calculating for Location ID: {location_id}')

        # Replace outliers with zero
        group_no_outliers, outliers, outliers_count = replace_outliers_w_zero(group)
        outliers_counts.append(outliers_count)
        groups.append(group)
        groups_no_outliers.append(group_no_outliers)

        # Calculate center point
        center_point, center_point_point = calculate_center_point(group_no_outliers)

        # Assign 'nearby' flag
        group['nearby'] = group.apply(lambda row: is_nearby(row['geometry'], center_point_point), axis=1)

        # Load data for center point and preprocess street network
        G, vegetation_natural, landuse_natural, park_natural, water_natural, wetland_natural, tree_natural, buildings_osm = load_data_for_center_point(center_point, distance*1.5)
        edges_gdf = preprocess_street_network(G)

        for index, row in group.iterrows():
            print(f'Processing data for Point {index} in {row["LocationID"]}')
            
            # Create data structures per point
            point_row = (row['latitude'], row['longitude'])
            bounding_box, bbox_coordinates = get_bounding_box(point_row, distance)
            scalebar_polygons, scale_coordinates = create_scale_bar(bbox_coordinates, point_row, meters=4)
            multipolygon_radius = create_radius_from_bbox(point_row, bounding_box, distance)
            # text = (f'point {index} in {row["LocationID"]}')
            # text_map = plt.text(bbox_coordinates[0], bbox_coordinates[2], text)

            # Check if the point is nearby
            if row['nearby']:
                # Plotting for nearby points
                plot_from_multiple_points(index, G, edges_gdf, buildings_osm, park_natural, landuse_natural,
                                          water_natural, wetland_natural, tree_natural, point_row,
                                          scalebar_polygons, multipolygon_radius, #text,
                                          fig_size=[10, 10], dpi=600, bbox=bbox_coordinates)
            else:
                # Load data for center point and preprocess street network for distant points
                center_point = point_row
                G_temp, vegetation_temp, landuse_temp, park_temp, water_temp, wetland_temp, tree_temp, buildings_osm_temp = load_data_for_center_point(center_point, distance)
                edges_gdf_temp = preprocess_street_network(G_temp)
                
                # Plotting for distant points
                plot_from_multiple_points(index, G_temp, edges_gdf_temp, buildings_osm_temp, park_temp, landuse_temp,
                                          water_temp, wetland_temp, tree_temp, point_row,
                                          scalebar_polygons, multipolygon_radius, #text,
                                          fig_size=[10, 10], dpi=600, bbox=bbox_coordinates)

    return outliers_counts, groups, groups_no_outliers


ssid_gdf_plotting01 = process_location_data(ssid_gdf, distance=100)